In [12]:
import os
import yaml
import importlib.util

def load_tools_from_registry(registry_path="tools/tool_registry.yaml"):
    """
    Loads all tools listed in the metadata registry and returns them as a list of callable functions.
    Assumes all tools are defined in Python files and decorated with @tool.
    """
    if not os.path.exists(registry_path):
        print(f"⚠️ Registry file not found: {registry_path}")
        return []

    with open(registry_path) as f:
        registry = yaml.safe_load(f) or []

    loaded_tools = []

    for entry in registry:
        tool_name = entry.get("name")
        tool_file = entry.get("saved_in")

        if not tool_name or not tool_file:
            print(f"❌ Invalid entry in registry: {entry}")
            continue

        # Convert path to module name
        module_name = tool_file.replace("/", ".").replace(".py", "")

        # Dynamically import the module
        try:
            spec = importlib.util.spec_from_file_location(module_name, tool_file)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)

            # Get the tool by name
            tool_fn = getattr(module, tool_name)
            loaded_tools.append(tool_fn)
            print(f"✅ Loaded tool: {tool_name}")

        except Exception as e:
            print(f"❌ Failed to load tool '{tool_name}' from '{tool_file}': {e}")

    return loaded_tools


In [13]:
# from load_tools_from_registry import load_tools_from_registry

# Load all generated tools
dynamic_tools = load_tools_from_registry()

# Register with agent or toolkit
# Example with OpenAI LangChain agent:
# agent = initialize_agent(tools=dynamic_tools + [search_tool, tool_generator], ...)


❌ Failed to load tool 'compound_interest_calculator' from 'tools/generated_tools.py': name 'tool' is not defined
❌ Failed to load tool 'calculate_bmi' from 'tools/generated_tools.py': name 'tool' is not defined


In [11]:
generated_tools

[]

In [3]:
import importlib
import tools.generated_tools

importlib.reload(tools.generated_tools)
generated_tools = load_generated_tools("tools.generated_tools")


NameError: name 'tool' is not defined